In [1]:
import sys
sys.path.append('../../../../')

In [18]:
%load_ext autoreload
%autoreload 2

import torch
from jtvae import JTVAE
from fast_jtnn.datautils import MolTreeFolder, PairTreeFolder, MolTreeDataset

path_adj = "./"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from rdkit import Chem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

In [22]:
from dig.ggraph.dataset import ZINC250k, ZINC800
from dig.ggraph.evaluation import RandGenEvaluator

In [23]:
dataset = ZINC800(one_shot=False, root=path_adj+'./datasets')

In [24]:
zinc_800_jt = torch.load(path_adj+"datasets/zinc_800_jt/processed/data.pt")

In [25]:
smiles = zinc_800_jt[-1]

In [37]:
# Define model
jtvae = JTVAE(smiles)
config = {"hidden_size": 420, "latent_size": 56, "depthT": 20, "depthG": 3}
jtvae.get_model('rand_gen', config)

/data/bora.oztekin/anaconda3/envs/jtvae37f/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [28]:
preprocessed = jtvae.preprocess(smiles)

100%|██████████| 800/800 [04:36<00:00,  2.90it/s] 


In [45]:
loader = MolTreeFolder(preprocessed, jtvae.vocab, 32, num_workers=4)
load_epoch = 0
lr = 1e-3
anneal_rate = 0.9
clip_norm = 50.0
num_epochs = 1
total_step = 0  # TODO args.load_epoch
beta = 0.0  # TODO args.beta
max_beta = 1.0
save_iter = 5000
step_beta = 0.002
anneal_iter = 40000
print_iter = 50
kl_anneal_iter = 2000
jtvae.train_rand_gen(loader, load_epoch, lr, anneal_rate, clip_norm, num_epochs, beta, max_beta, step_beta, anneal_iter, kl_anneal_iter, print_iter, save_iter)

Model #Params: 4260K


In [46]:
samples = list(map(Chem.MolFromSmiles, jtvae.run_rand_gen(30)))

In [47]:
res_dict = {'mols': samples, 'train_smiles': smiles}
evaluator = RandGenEvaluator()
results = evaluator.eval(res_dict)
print(results)

Valid Ratio: 30/30 = 100.00%
Unique Ratio: 17/30 = 56.67%
Novel Ratio: 30/30 = 100.00%
{'valid_ratio': 100.0, 'unique_ratio': 56.666666666666664, 'novel_ratio': 100.0}
